<a href="https://colab.research.google.com/github/Nayan-phero/Nayan-phero/blob/main/Job_data_insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [19]:
#Relevant Libraries Import

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import os
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import joblib
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

#import shap
#%matplotlib inline
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
#from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score , recall_score, precision_score, f1_score


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


import seaborn as sns

from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn. preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import xgboost
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier


# **Combine section**

# ** Batc_8 data**

**2nd dataset**

In [20]:
balanced_full_data = pd.read_csv('/content/Filtered_Final_Merged_Data.csv')


In [21]:
balanced_full_data.head(5)

,Email,quiz_1,quiz_2,quiz_3,quiz_4,quiz_5,assignment_1,assignment_2,assignment_3,assignment_4,...,Week 5 points,"Week 6,7 points",Week 8 points,"Obtained points from Scrum, presentation and session (102 points)",Obtained points from Video resume (10 points),Obtained points from texted resume (10 points),Obtained points from Individual documentation (20 points),Obtained marks from technical evaluation (30 points),Manager's bonus (6 points),Grand Total
0,sakibsarker6969@gmail.com,0.933333,NaN,NaN,NaN,NaN,0.966667,1.000000,1.000000,0.916667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,abusiddik9994@gmail.com,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,0.883333,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,badhonsarker38@gmail.com,0.933333,NaN,NaN,NaN,NaN,1.000000,1.000000,0.766667,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aristoxcel@gmail.com,0.733333,NaN,NaN,NaN,NaN,0.966667,0.883333,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,afrantarek123456789@gmail.com,1.000000,NaN,NaN,NaN,NaN,1.000000,0.750000,1.000000,0.883333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:

columns_to_drop = ['Email', 'quiz_5', 'job_title', 'created_at','Name','Student ID','currently_working_there']
balanced_full_data = balanced_full_data.drop(columns=columns_to_drop, errors='ignore')

In [23]:
import plotly.express as px
import pandas as pd

def plot_interactive_trends(data, feature):
    # Count hired and not-hired students
    hired_counts = data[data['is_got_hired'].str.lower() == 'yes'][feature].value_counts()
    not_hired_counts = data[data['is_got_hired'].str.lower() == 'no'][feature].value_counts()

    # Combine counts into a DataFrame
    demographic_data = pd.DataFrame({
        'Category': hired_counts.index.tolist() + not_hired_counts.index.tolist(),
        'Count': list(hired_counts) + list(not_hired_counts),
        'Hiring Status': ['Hired'] * len(hired_counts) + ['Not Hired'] * len(not_hired_counts)
    }).fillna(0)

    # Calculate total students and hiring ratio
    demographic_data['Total Students'] = demographic_data.groupby('Category')['Count'].transform('sum')
    demographic_data['Hiring Ratio (%)'] = 0.0  # Initialize as float
    for category in demographic_data['Category'].unique():
        total_count = demographic_data[demographic_data['Category'] == category]['Total Students'].iloc[0]
        hired_count = demographic_data[
            (demographic_data['Category'] == category) & (demographic_data['Hiring Status'] == 'Hired')
        ]['Count'].iloc[0] if not demographic_data[
            (demographic_data['Category'] == category) & (demographic_data['Hiring Status'] == 'Hired')
        ]['Count'].empty else 0
        demographic_data.loc[demographic_data['Category'] == category, 'Hiring Ratio (%)'] = (
            (hired_count / total_count * 100) if total_count > 0 else 0
        )

    # Add total students and hiring ratio as text labels
    demographic_data['Text'] = demographic_data.apply(
        lambda row: f"Total: {int(row['Total Students'])}\nRatio: {row['Hiring Ratio (%)']:.1f}%", axis=1
    )

    # Create the interactive bar chart
    fig = px.bar(
        demographic_data,
        x='Category',
        y='Count',
        color='Hiring Status',
        title=f'{feature.capitalize()} Distribution of Hired vs. Not Hired Students',
        barmode='group',
        labels={'Category': feature.capitalize(), 'Count': 'Number of Students'},
        text='Text'  # Use the 'Text' column for labels
    )
    fig.update_traces(textposition='outside')
    fig.show()

# List of features to analyze
specific_features = [
    'location_type', 'gender', 'age', 'device_type', 'internet_type',
    'want_job', 'have_job_experience',
    'education_level', 'related_to_it', 'category'
]

# Generate visualizations for each feature
for feature in specific_features:
    plot_interactive_trends(balanced_full_data, feature)


In [24]:
rename_mapping = {
    "quiz_1": "Quiz_101_GitHub_Branding",
    "quiz_2": "Quiz_201_Resume_Description",
    "quiz_3": "Quiz_301_Mock_Interview",
    "quiz_4": "Cover_Letter_Quiz",
    "assignment_1": "GitHub_Profile_Setup",
    "assignment_2": "Personal_Branding_Doc",
    "assignment_3": "Portfolio_Submission",
    "assignment_4": "Quiz_101_Presentation",
    "assignment_5": "English_Speaking_Writing",
    "assignment_6": "Email_Writing_Tips",
    "assignment_7": "Resume_Building",
    "assignment_8": "Cover_Letter_Writing",
    "assignment_9": "Frontend_Development",
    "assignment_10": "Fullstack_Development"
}

# Apply the renaming to the DataFrame
balanced_full_data.rename(columns=rename_mapping, inplace=True)




In [26]:
import plotly.express as px
import pandas as pd

def plot_interactive_trends(data, feature):
    # Count hired and not-hired students
    hired_counts = data[data['is_got_hired'].str.lower() == 'yes'][feature].value_counts()
    not_hired_counts = data[data['is_got_hired'].str.lower() == 'no'][feature].value_counts()

    # Combine counts into a DataFrame
    demographic_data = pd.DataFrame({
        'Category': hired_counts.index.tolist() + not_hired_counts.index.tolist(),
        'Count': list(hired_counts) + list(not_hired_counts),
        'Hiring Status': ['Hired'] * len(hired_counts) + ['Not Hired'] * len(not_hired_counts)
    }).fillna(0)

    # Calculate total students and hiring ratio
    demographic_data['Total Students'] = demographic_data.groupby('Category')['Count'].transform('sum')
    demographic_data['Hiring Ratio (%)'] = 0.0  # Initialize as float
    for category in demographic_data['Category'].unique():
        total_count = demographic_data[demographic_data['Category'] == category]['Total Students'].iloc[0]
        hired_count = demographic_data[
            (demographic_data['Category'] == category) & (demographic_data['Hiring Status'] == 'Hired')
        ]['Count'].iloc[0] if not demographic_data[
            (demographic_data['Category'] == category) & (demographic_data['Hiring Status'] == 'Hired')
        ]['Count'].empty else 0
        demographic_data.loc[demographic_data['Category'] == category, 'Hiring Ratio (%)'] = (
            (hired_count / total_count * 100) if total_count > 0 else 0
        )

    # Add total students and hiring ratio as text labels
    demographic_data['Text'] = demographic_data.apply(
        lambda row: f"Total: {int(row['Total Students'])}\nRatio: {row['Hiring Ratio (%)']:.1f}%", axis=1
    )

    # Create the interactive bar chart
    fig = px.bar(
        demographic_data,
        x='Category',
        y='Count',
        color='Hiring Status',
        title=f'{feature.capitalize()} Distribution of Hired vs. Not Hired Students',
        barmode='group',
        labels={'Category': feature.capitalize(), 'Count': 'Number of Students'},
        text='Text'  # Use the 'Text' column for labels
    )
    fig.update_traces(textposition='outside')
    fig.show()

# List of features to analyze
specific_features = [
    "Quiz_101_GitHub_Branding",
    "Quiz_201_Resume_Description",
    "Quiz_301_Mock_Interview",
    "Cover_Letter_Quiz",
    "Quiz_101_Presentation",
    "English_Speaking_Writing",
    "Frontend_Development",
    "Fullstack_Development",
    "interview_mark_in_percent"
]

# Generate visualizations for each feature
for feature in specific_features:
    plot_interactive_trends(balanced_full_data, feature)
